In [1]:
import numpy as np
import numba
from numba import cuda, njit
import cupy as cp
from math import floor

In [ ]:
alpha = 2
delta_x = 1
delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)
size = 10000
nthreads = 32 #wątki w warpie
nblocksy = (size + nthreads - 1) // nthreads  #blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock ;
nblocksx = (size + nthreads - 1) // nthreads

original_array = np.zeros((size, size), dtype = np.float64)
original_array[0 ,:]= 100

In [ ]:
def cpu_calculate(u):
  newArray = u.copy()
  for i in range(1, size-1, delta_x):
      for j in range(1, size-1, delta_x):
          val = (gamma * (u[i+1][j] + u[i-1][j] + u[i][j+1] + u[i][j-1] - 4*u[i][j]) + u[i][j])
          newArray[i, j] = floor(val * 1e3) / 1e3
  return newArray

In [ ]:
def start_cpu(iter):
  start_cpu = cp.cuda.Event()
  end_cpu = cp.cuda.Event()
  start_cpu.record()
  cpu_array = np.copy(original_array)
  for i in range(iter):
    cpu_array = cpu_calculate(cpu_array)
  end_cpu.record()
  end_cpu.synchronize()
  t_cpu = cp.cuda.get_elapsed_time(start_cpu, end_cpu)
  return cpu_array,t_cpu

In [ ]:
@njit
def cpu_calculate_numba(u):
    newArray = u.copy()
    for i in range(1, size-1, delta_x):
        for j in range(1, size-1, delta_x):
            val = (gamma * (u[i+1, j] + u[i-1, j] + u[i, j+1] + u[i, j-1] - 4*u[i, j]) + u[i, j])
            newArray[i, j] = floor(val * 1e3) / 1e3
    return newArray

In [ ]:
def start_cpu_numba(iter):
  start_cpu_numba = cp.cuda.Event()
  end_cpu_numba = cp.cuda.Event()
  start_cpu_numba.record()
  cpu_array_numba = np.copy(original_array)
  for i in range(iter):
    cpu_array_numba = cpu_calculate_numba(cpu_array_numba)
  end_cpu_numba.record()
  end_cpu_numba.synchronize()
  t_cpu_numba = cp.cuda.get_elapsed_time(start_cpu_numba, end_cpu_numba)
  return cpu_array_numba,t_cpu_numba

In [ ]:
cpu_array_numba, t_cpu = start_cpu_numba(100)
gpu_arr, gpu_t = start_parallel_gpu(100)

In [ ]:
if (cp.array_equal(cpu_array_numba, gpu_arr)):
  print("Same Arrays")
else:
  print("Different CPU/GPU Arrays")

Same Arrays


In [ ]:
@cuda.jit
def parallel_gpu(u,gpu_new_array_par):
  i, j = cuda.grid(2)
  if ((i > 0 and i < size-1) and (j > 0 and j < size-1)):
    val = (gamma * (u[i+1][j] + u[i-1][j] + u[i][j+1] + u[i][j-1] - 4*u[i][j]) + u[i][j])
    gpu_new_array_par[i][j] = floor(val * 1e3) / 1e3

In [ ]:
def start_parallel_gpu(iter):
  start_parallel_gpu = cp.cuda.Event()
  end_parallel_gpu = cp.cuda.Event()
  start_parallel_gpu.record()
  d_a = numba.cuda.to_device(original_array)
  d_b = numba.cuda.to_device(original_array)
  for i in range(0,iter):
        parallel_gpu[(nblocksx, nblocksy), (nthreads, nthreads)](
          d_a, d_b
        )
        d_a.copy_to_device(d_b)
  gpu_result = d_a.copy_to_host()
  end_parallel_gpu.record()
  end_parallel_gpu.synchronize()
  t_gpu = cp.cuda.get_elapsed_time(start_parallel_gpu, end_parallel_gpu)
  return gpu_result, t_gpu

In [ ]:
def comp(cpu_arr, cpu_time, cpu_arr_numba, cpu_time_numba, gpu_arr, gpu_time):
  cpu_time_s = cpu_time/1000
  cpu_time_numba_s = cpu_time_numba/1000
  gpu_time_s = gpu_time/1000

  if (cp.array_equal(cpu_arr, cpu_arr_numba)):
    if (cp.array_equal(cpu_arr, gpu_arr)):
      print("Same Arrays")
    else:
      print("Different CPU/GPU Arrays")
  else:
    print("Different CPU Arrays")

    print(cpu_arr)
    print("________")
    print(cpu_arr_numba)
    print("________")
    print(gpu_arr)

  if cpu_time < cpu_time_numba and cpu_time < gpu_time:
    print("CPU was faster")
    print('time: %fs' % (cpu_time_s))
    print(f'vs Numba time: {cpu_time_numba_s:.3f}s, difference {cpu_time_s-cpu_time_numba_s:.3f}s, speedup x{cpu_time_numba_s/cpu_time_s:.3f}')
    print(f'vs CUDA time: {gpu_time_s:.3f}s, difference {cpu_time_s-gpu_time_s:.3f}s, speedup x{gpu_time_s/cpu_time_s:.3f}')
  if cpu_time_numba < cpu_time and cpu_time_numba < gpu_time:
    print("Numba was faster")
    print('time: %fs' % (cpu_time_numba_s))
    print(f'vs CPU time: {cpu_time_s:.3f}s, difference {cpu_time_numba_s-cpu_time_s:.3f}s, speedup x{cpu_time_s/cpu_time_numba_s:.3f}')
    print(f'vs CUDA time: {gpu_time_s:.3f}s, difference {cpu_time_numba_s-gpu_time_s:.3f}s, speedup x{gpu_time_s/cpu_time_numba_s:.3f}')
  if gpu_time < cpu_time and gpu_time < cpu_time_numba:
    print("CUDA was faster")
    print('time: %fs' % (gpu_time_s))
    print(f'vs CPU time: {cpu_time_s:.3f}s, difference {gpu_time_s-cpu_time_s:.3f}s, speedup x{cpu_time_s/gpu_time_s:.3f}')
    print(f'vs Numba time: {cpu_time_numba_s:.3f}s, difference {gpu_time_s-cpu_time_numba_s:.3f}s, speedup x{cpu_time_numba_s/gpu_time_s:.3f}')


In [ ]:
def main(iter):
  cpu_arr, cpu_t = start_cpu(iter)
  cpu_numba_arr, cpu_numba_t = start_cpu_numba(iter)
  gpu_arr, gpu_t = start_parallel_gpu(iter)
  comp(cpu_arr, cpu_t, cpu_numba_arr, cpu_numba_t, gpu_arr, gpu_t)

In [ ]:
main(5)

Same Arrays
CUDA was faster
time: 0.010768s
vs CPU time: 12.706s, difference -12.695s, speedup x1179.967
vs Numba time: 0.032s, difference -0.021s, speedup x2.928


In [ ]:
main(100)

Same Arrays
CUDA was faster
time: 0.096003s
vs CPU time: 259.184s, difference -259.088s, speedup x2699.762
vs Numba time: 0.551s, difference -0.455s, speedup x5.735


In [ ]:
main(200)

Same Arrays
CUDA was faster
time: 0.184062s
vs CPU time: 513.208s, difference -513.024s, speedup x2788.235
vs Numba time: 1.045s, difference -0.861s, speedup x5.676


In [ ]:
main(500)

Same Arrays
CUDA was faster
time: 0.354552s
vs CPU time: 1264.504s, difference -1264.150s, speedup x3566.481
vs Numba time: 2.611s, difference -2.257s, speedup x7.365


In [ ]:
main(1000)

Same Arrays
CUDA was faster
time: 0.528386s
vs CPU time: 2488.992s, difference -2488.463s, speedup x4710.553
vs Numba time: 5.109s, difference -4.581s, speedup x9.670


# Profiler


In [ ]:
%%writefile python_cuda_profile.py
import numpy as np
import numba
from numba import cuda, njit
import cupy as cp
import cupyx.profiler
from math import floor
alpha = 2
delta_x = 1
delta_t = (delta_x ** 2)/(4 * alpha)
gamma = (alpha * delta_t) / (delta_x ** 2)
size = 1000
nthreads = 32
nblocksy = (size + nthreads - 1) // nthreads
nblocksx = (size + nthreads - 1) // nthreads

original_array = np.zeros((size, size), dtype = np.float64)
original_array[0 ,:]= 100

@cuda.jit
def parallel_gpu(u,gpu_new_array_par):
  i, j = cuda.grid(2)
  if ((i > 0 and i < size-1) and (j > 0 and j < size-1)):
    val = (gamma * (u[i+1][j] + u[i-1][j] + u[i][j+1] + u[i][j-1] - 4*u[i][j]) + u[i][j])
    gpu_new_array_par[i][j] = floor(val * 1e3) / 1e3

def start_parallel_gpu(iter):
  start_parallel_gpu = cp.cuda.Event()
  end_parallel_gpu = cp.cuda.Event()
  start_parallel_gpu.record()
  d_a = numba.cuda.to_device(original_array)
  d_b = numba.cuda.to_device(original_array)
  for i in range(0,iter):
        parallel_gpu[(nblocksx, nblocksy), (nthreads, nthreads)](
          d_a, d_b
        )
        d_a.copy_to_device(d_b)
  gpu_result = d_a.copy_to_host()
  end_parallel_gpu.record()
  end_parallel_gpu.synchronize()
  t_gpu = cp.cuda.get_elapsed_time(start_parallel_gpu, end_parallel_gpu)
  return gpu_result, t_gpu

def main(iter):
  with cupyx.profiler.profile():
    gpu_arr, gpu_t = start_parallel_gpu(iter)

main(500)

Overwriting python_cuda_profile.py


In [ ]:
!nvprof --openacc-profiling off python ./python_cuda_profile.py

==26212== NVPROF is profiling process 26212, command: python3 ./python_cuda_profile.py
==26212== Profiling application: python3 ./python_cuda_profile.py
==26212== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   87.85%  305.95ms       500  611.90us  349.40us  855.70us  _ZN6cudapy8__main__12parallel_gpuB2v1B106cw51cXTLSUwHBinCqbbgUAAGBlq82ILSCEQYkgSQBFCjFSaBZJtttTo4sahbKRjoKKiDvAVKN0AuNEDVYWGDJATUC_2bQZ12oCAA_3d_3dE5ArrayIdLi2E1C7mutable7alignedE5ArrayIdLi2E1C7mutable7alignedE
                   10.46%  36.429ms       500  72.858us  69.950us  75.871us  [CUDA memcpy DtoD]
                    1.04%  3.6326ms         2  1.8163ms  1.0999ms  2.5327ms  [CUDA memcpy HtoD]
                    0.65%  2.2722ms         1  2.2722ms  2.2722ms  2.2722ms  [CUDA memcpy DtoH]
      API calls:   62.52%  383.92ms         1  383.92ms  383.92ms  383.92ms  cudaProfilerStart
                   20.94%  128.62ms         1  128.62ms  128.62ms 